In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
def hist_vol(X):
    X_resampled = pd.DataFrame()
    # X_resampled['prev_month_volatility'] = X['Close'].resample('M').std()
    # X_resampled['prev2_month_volatilty'] = X['Close'].resample('M').std().shift(1)
    # X_resampled['vol_ratio'] = X_resampled['prev_month_volatility']/X_resampled['prev2_month_volatilty']

    X_resampled['vol_ratio'] = X['Close'].resample('M').std()/X['Close'].resample('M').std().shift(1)
    return X_resampled

def log_range(X):
    X_resampled = pd.DataFrame()
    X_resampled['log_high_low'] = np.log(X['Close'].resample('M').max()) - np.log(X['Close'].resample('M').min())
    return X_resampled

In [ ]:
class log_change(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X['log_close_diff'] = np.log(X['Close']) - np.log(X['Close'].shift(1))
        return X

class pct_change(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X['1month_return'] = X['Close']/X['Close'].shift(1) - 1
        return X

## Workspace

In [ ]:
def create_feature(daily_df):
    daily_df.index = pd.to_datetime(daily_df.index)
    monthly_df = daily_df.resample('M').last()

    # add new daily features here
    daily_features = pd.concat([log_range(daily_df),hist_vol(daily_df)],axis=1)
    
    monthly_cols = monthly_df.columns

    # add new class or functions if needed
    baseline_pipeline = Pipeline([
            # ('generate_class', generate_class()),
            ('log_change', log_change()),
            ('pct_change', pct_change())
    ])
    
    baseline_df = baseline_pipeline.fit_transform(monthly_df)
    baseline_df = baseline_df.drop(monthly_cols,axis=1)

    return_df = pd.concat([baseline_df, daily_features],axis=1)
    
    # generate signal
    return_ = (1+daily_df.pct_change()).resample('M').prod()-1
    return_df['Signal'] = np.where(return_>ub, 2, np.where(return_<lb, 0, 1))
    return_df['Signal'] = return_df['Signal'].shift(1)

    return return_df

In [ ]:
# read csv
daily_df = pd.read_csv("Portfolio_data/Market_portfolio_daily.csv",index_col=0)
manufacture_daily = pd.read_csv('Portfolio_data/MANUFACTURING_portfolio_daily.csv',index_col=0)
multi_portfolio_daily = pd.read_csv('Portfolio_data/MULTIPLE_portfolio_daily.csv',index_col=0)
tech_daily = pd.read_csv('Portfolio_data/TECHNOLOGY_portfolio_daily.csv',index_col=0)
TS_daily = pd.read_csv('Portfolio_data/TRADE & SERVICES_portfolio_daily.csv',index_col=0)

In [ ]:
# create features by calling the function

create_feature(manufacture_daily).to_csv('Intermediate_data/market_baseline_v1.csv')
create_feature(manufacture_daily).to_csv('Intermediate_data/manufacture_baseline_v1.csv')
create_feature(multi_portfolio_daily).to_csv('Intermediate_data/multi_portfolio_baseline_v1.csv')
create_feature(tech_daily).to_csv('Intermediate_data/tech_baseline_v1.csv')
create_feature(TS_daily).to_csv('Intermediate_data/trade&service_baseline_v1.csv')